In [ ]:
import sqlite3

from IPython.core.magic import register_cell_magic

from tabulate import tabulate
import sqlparse

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()


def run_and_print(sql):
    try:
        cursor.execute(sql)
    except sqlite3.OperationalError as e:
        print("Error:\n", sql)
        raise e

    conn.commit()

    rows = cursor.fetchall()
    if cursor.description is None:
        return

    print(tabulate(rows, headers=[d[0] for d in cursor.description]), end='\n\n')


@register_cell_magic
def sql(_, cell):
    run_and_print("DROP TABLE IF EXISTS trace")
    run_and_print("CREATE TABLE trace(trace TEXT)")
    for sql in sqlparse.split(cell):
        run_and_print(sql)
    run_and_print("SELECT * FROM trace")

In [ ]:
%%sql

INSERT INTO trace VALUES ('Reticulating splines...');

CREATE TABLE signal(
    clk BOOLEAN
    );
INSERT INTO signal VALUES (
    FALSE
    );

CREATE TABLE register(
    R0 INT,
    R1 INT,
    R2 INT,
    R3 INT,
    R4 INT,
    R5 INT,
    R6 INT,
    R7 INT,
    PC INT,
    COND INT
    );

INSERT INTO register VALUES (
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    );

CREATE TABLE memory(
    address INT,
    value INT
    );

pragma recursive_triggers = 1;


In [ ]:
%%sql

DROP TRIGGER IF EXISTS clk_trigger;
CREATE TRIGGER clk_trigger
AFTER UPDATE OF clk ON signal
BEGIN
    INSERT INTO trace VALUES (
        CASE
            WHEN OLD.clk = 0 AND NEW.clk = 1 THEN 'rising edge'
            WHEN OLD.clk = 1 AND NEW.clk = 0 THEN 'falling edge'
            ELSE 'no edge detected'  -- Optional, for other cases
        END
    );
END;

In [ ]:
%%sql
-- drive the clock signal
UPDATE signal SET clk = 1;
UPDATE signal SET clk = 0;
UPDATE signal SET clk = 1;
UPDATE signal SET clk = 0;
UPDATE signal SET clk = 0;
UPDATE signal SET clk = 1;
UPDATE signal SET clk = 1;
UPDATE signal SET clk = 0;
UPDATE signal SET clk = 1;